In [1]:
%run multitype_model.ipynb
%run data_dealing.ipynb
%run train.ipynb
#上面这俩等训练时候在用，现在用不上
from transformers import AutoConfig, AutoModel
#读取ipynb文件，里面最好只有类和函数
import warnings
import random
warnings.filterwarnings("ignore")    #去掉warning提示

/home/jiaoyilin/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.set_device(1)

liuyan_data=data_deal_liuyan('new投诉数据(1000条).xlsx')   #最长的投诉还不到1000个字，就这！？
random.shuffle(liuyan_data)
train_feature=liuyan_data[:800]
dev_feature=liuyan_data[800:900]
test_feature=liuyan_data[900:]

In [3]:
data = data_dealing('政府回应编码（1000条）.xlsx')   
random.shuffle(data)    #打乱数据以方便划分
train_feature = data[:800]
dev_feature = data[800:900]
test_feature = data[900:]
print(len(train_feature),len(dev_feature),len(test_feature))

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f4ac0197280>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: e7d24be7-8718-4dad-8497-54a4281ea69d)')' thrown while requesting HEAD https://huggingface.co/IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese/resolve/main/tokenizer_config.json


800 100 100


In [4]:
model_config = AutoConfig.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese')
model = AutoModel.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese')

loss = torch.nn.CrossEntropyLoss()
my_model = bert_model(model_config,model,loss)
#模型测试完毕，可以运行，然后就是训练了
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device = device 
my_model.to(device)
#print(len(train_feature),len(dev_feature),len(test_feature))
#get_avaliable_memory(device)
#my_model.load_state_dict(torch.load('huifu_model.pth'))

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f4ac018e2b0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 27113e36-c74c-47c4-a59c-f2ef54bf4d10)')' thrown while requesting HEAD https://huggingface.co/IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese/resolve/main/config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f4ac018e580>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: bb61914c-28f1-4ff9-ba11-6ca3f2851381)')' thrown while requesting HEAD https://huggingface.co/IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese/resol

bert_model(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(12800, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
              (dropo

In [5]:
%%time
#模型训练以及结果检查
F1_score=[]
for i in range(2):    #测试2次取平均结果
    train(my_model,train_feature,dev_feature,2,7,2e-4,device)    
    test_model =bert_model(model_config, model,loss)
    test_model.to(device)
    test_model.load_state_dict(torch.load('huifu_model.pth',map_location='cpu'))
    test_F1 = evaluate(test_model, test_feature,device, "test")
#训练了14轮，验证集74，测试集75，回复类型分类
#训练了10轮，验证集87，测试集85，留言内容分类

已读取的数据数： 0
已读取的数据数： 100
已读取的数据数： 200
已读取的数据数： 300
已读取的数据数： 400
已读取的数据数： 500
已读取的数据数： 600
已读取的数据数： 700
第 0 轮loss为: tensor(1.0490, device='cuda:1', grad_fn=<NllLossBackward0>)
dev数据准确率为: 0.74 数据总数: 100 判断对的数目: 74
已读取的数据数： 0
已读取的数据数： 100
已读取的数据数： 200
已读取的数据数： 300
已读取的数据数： 400
已读取的数据数： 500
已读取的数据数： 600
已读取的数据数： 700
第 1 轮loss为: tensor(4.7665, device='cuda:1', grad_fn=<NllLossBackward0>)
dev数据准确率为: 0.71 数据总数: 100 判断对的数目: 71
已读取的数据数： 0
已读取的数据数： 100
已读取的数据数： 200
已读取的数据数： 300
已读取的数据数： 400
已读取的数据数： 500
已读取的数据数： 600
已读取的数据数： 700
第 2 轮loss为: tensor(0.0926, device='cuda:1', grad_fn=<NllLossBackward0>)
dev数据准确率为: 0.76 数据总数: 100 判断对的数目: 76
已读取的数据数： 0
已读取的数据数： 100
已读取的数据数： 200
已读取的数据数： 300
已读取的数据数： 400
已读取的数据数： 500
已读取的数据数： 600
已读取的数据数： 700
第 3 轮loss为: tensor(1.2735, device='cuda:1', grad_fn=<NllLossBackward0>)
dev数据准确率为: 0.73 数据总数: 100 判断对的数目: 73
已读取的数据数： 0
已读取的数据数： 100
已读取的数据数： 200
已读取的数据数： 300
已读取的数据数： 400
已读取的数据数： 500
已读取的数据数： 600
已读取的数据数： 700
第 4 轮loss为: tensor(6.4311e-05, device='cuda:1', grad_f

In [8]:
#test_model =bert_model(model_config, model,loss)
#test_model.to(device)
my_model.load_state_dict(torch.load('huifu_model.pth',map_location='cpu'))
test_F1 = evaluate(my_model, test_feature,device, "test")

test数据准确率为: 0.17708333333333334 数据总数: 96 判断对的数目: 17


In [5]:
72/96

0.75